# Bias Uncovering Test Case (BTC) for Gender Bias

In [2]:
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc

## BiasFinder on IMDB

In [69]:
mutation_tool = "biasfinder"
# mutation_tool = "eec"
bias_type = "gender"
# task = "imdb" # dataset used for fine-tuning
mutant = "imdb" # dataset used for generating mutants
# mutant = "twitter_semeval" # dataset used for generating mutants
# mutant = "twitter_s140" # dataset used for generating mutants

def load_prediction(fpath):
    with open(fpath, 'rb') as f:
        pred = pickle.load(f)
    return pred

def load_mutant_and_prediction(mutation_tool, bias_type, mutant):
    base_dir = f"../../data/{mutation_tool}/{bias_type}/{mutant}/"
    if mutation_tool == "biasfinder" :
        df = pd.read_csv(base_dir + "test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "gender"])
    elif mutation_tool == "eec":
        df = pd.read_csv(base_dir + "test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "person", "gender", "emotion"])

    df["template"] = df["template"].astype("category")
    df["template_id"] = df["template"].cat.codes

    prediction_fpath = os.path.join(base_dir, "prediction.pkl")
    pred = load_prediction(prediction_fpath)

    df["prediction"] = pred

    return df


df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)

df.head()


,label,mutant,template,original,gender,template_id,prediction
0,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",male,929,0
1,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",female,929,0
2,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1
3,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1
4,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1


In [70]:
print("Accuracy: {:.2f}%".format(100*sum(df["label"] == df["prediction"])/len(df["label"])))


Accuracy: 93.24%


In [71]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

# Mutants 	: 153866
# Templates 	: 3015
# BTCs 		: 3198


## BTC Comparison with EEC on Twitter SemEval fine-tuned Model

### EEC Mutants

In [72]:
df = load_mutant_and_prediction(mutation_tool="eec", bias_type="gender", mutant="twitter_semeval")
evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)


# Mutants 	: 8400
# Templates 	: 140
# BTCs 		: 1868


### BiasFinder Mutants generated from SemEval test set

In [73]:
df = load_mutant_and_prediction(
    mutation_tool="biasfinder", bias_type="gender", mutant="twitter_semeval")
evaluation = evaluate_btc(df["label"], df["prediction"],
                          df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)


# Mutants 	: 386
# Templates 	: 9
# BTCs 		: 1378


### BiasFinder Mutants generated from 10k train data from Twitter S140

In [74]:
df = load_mutant_and_prediction(
    mutation_tool="biasfinder", bias_type="gender", mutant="twitter_s140")
evaluation = evaluate_btc(df["label"], df["prediction"],
                          df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)


# Mutants 	: 58388
# Templates 	: 1672
# BTCs 		: 27114
